# CrabNet Hyperparameter Surrogate Model

Here, we train a surrogate model for the CrabNet hyperparameter optimization on Matbench
datasets. We capture average model performance (MAE and RMSE) as well as model
complexity and runtime.

In [38]:
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold, GroupKFold
import pandas as pd
import numpy as np
import joblib
from os import path
import json

# attempted use of skl2onnx to convert to onnx failing due to protobuf error
# https://github.com/onnx/onnx/issues/4469

# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType

In [39]:
dummy = True

task_name = "crabnet_hyperparameter"

data_dir = path.join("..", "..", "data", "processed", task_name)
model_dir = path.join("..", "..", "models", task_name)

if dummy:
    model_dir = path.join(model_dir, "dummy")

## Import Data

In [40]:
sobol_reg = pd.read_csv(path.join(data_dir, "sobol_regression.csv"))

if dummy:
    sobol_reg = sobol_reg.head(100)

### Prepare data for Random Forest Classifier -- Convert Categorical Data to Numerical Data

In [41]:
bias_ohe = pd.get_dummies(sobol_reg["bias"], prefix="bias")
criterion_ohe = pd.get_dummies(sobol_reg["criterion"], prefix="criterion")
elemprop_ohe = pd.get_dummies(sobol_reg["elem_prop"], prefix="elem_prop")
hardware_ohe = pd.get_dummies(sobol_reg["hardware"], prefix="hardware")

sobol_reg = pd.concat([sobol_reg, bias_ohe, criterion_ohe, elemprop_ohe, hardware_ohe], axis=1)

sobol_reg.drop(columns=["bias", "criterion", "elem_prop", "hardware"], inplace=True)

In [42]:
sobol_reg.columns

Index(['_id', 'session_id', 'timestamp', 'N', 'alpha', 'd_model',
       'dim_feedforward', 'dropout', 'emb_scaler', 'eps', 'epochs_step',
       'fudge', 'heads', 'k', 'lr', 'pe_resolution', 'ple_resolution',
       'pos_scaler', 'weight_decay', 'batch_size', 'out_hidden4', 'betas1',
       'betas2', 'train_frac', 'model_size', 'runtime', 'runtime.1', 'mae',
       'mae.1', 'rmse', 'rmse.1', 'bias_False', 'bias_True',
       'criterion_RobustL1', 'criterion_RobustL2', 'elem_prop_magpie',
       'elem_prop_mat2vec', 'elem_prop_onehot', 'hardware_2080ti'],
      dtype='object')

## define f(x) to calc mae scores 

In [43]:
# argument for rfr_mae, X_array, y_array, model_name to save model as .pkl
def rfr_group_mae(X_array, y_array, group_array, model_name_stem, objective_name, random_state=13):
    kf = GroupKFold(n_splits=5)
    mae_scores = []
    y_preds = []
    y_trues = []
    for i, (train_index, test_index) in enumerate(
        kf.split(X_array, y_array, group_array)
    ):
        X_train, X_test = X_array[train_index], X_array[test_index]
        y_train, y_test = y_array[train_index], y_array[test_index]
        y_test = y_test.tolist()

        model = RandomForestRegressor(random_state=random_state)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test).tolist()

        y_preds.append(y_pred)
        y_trues.append(y_test)
        mae = mean_absolute_error(y_test, y_pred)
        mae_scores.append(mae)
        # save model as .pkl
        joblib.dump(model, f"{model_name_stem}_{i}.pkl")

    avg_mae = np.mean(mae_scores)
    std_mae = np.std(mae_scores)
    
    print(f"MAE for {objective_name}: {avg_mae:.4f} +/- {std_mae:.4f}")
    results = {"mae": mae_scores, "y_pred": y_preds, "y_true": y_trues}
    return results

## Features

In [44]:
common_features = [
    "N",
    "alpha",
    "d_model",
    "dim_feedforward",
    "dropout",
    "emb_scaler",
    "eps",
    "epochs_step",
    "fudge",
    "heads",
    "k",
    "lr",
    "pe_resolution",
    "ple_resolution",
    "pos_scaler",
    "weight_decay",
    "batch_size",
    "out_hidden4",
    "betas1",
    "betas2",
    "train_frac",
    "bias_False",
    "bias_True",
    "criterion_RobustL1",
    "criterion_RobustL2",
    "elem_prop_magpie",
    "elem_prop_mat2vec",
    "elem_prop_onehot",
    "hardware_2080ti"
]

# fba_isna_prob_features = common_features
# ls_isna_prob_features = common_features

# fba_features = common_features + ["fba_rank"]
# ls_features = common_features + ["ls_rank"]

# fba_time_s_features = common_features + ["fba_time_s_rank"]
# ls_time_s_features = common_features + ["ls_time_s_rank"]

## Objective Surrogate Models
#### no NaN values
### mae

In [45]:
mae_features = common_features + ["mae.1"]  ## ToDo: mae or mae_rank (mae.1)

X_array_mae = sobol_reg[mae_features].to_numpy()
y_array_mae = sobol_reg[["mae"]].to_numpy().ravel()

sobol_reg_mae_group = (
    sobol_reg[mae_features]
    .round(6)
    .apply(lambda row: "_".join(row.values.astype(str)), axis=1)
)

mae_model_stem = path.join(model_dir, "sobol_reg_mae")
mae_results = rfr_group_mae(X_array_mae, y_array_mae, sobol_reg_mae_group, mae_model_stem, "mae")

MAE for mae: 0.0883 +/- 0.0160


### rmse

In [46]:
rmse_features = common_features + ["rmse.1"] ## ToDo: rmse or rmse_rank (rmse.1)

X_array_rmse = sobol_reg[rmse_features].to_numpy()
y_array_rmse = sobol_reg[["rmse"]].to_numpy().ravel()

sobol_reg_rmse_group = (
    sobol_reg[rmse_features]
    .round(6)
    .apply(lambda row: "_".join(row.values.astype(str)), axis=1)
)

rmse_model_stem = path.join(model_dir, "sobol_reg_rmse")
rmse_results = rfr_group_mae(
    X_array_rmse, y_array_rmse, sobol_reg_rmse_group, rmse_model_stem, "rmse"
)

MAE for rmse: 0.0894 +/- 0.0248


### model size ###

In [47]:
model_size_features = common_features 

X_array_model_size = sobol_reg[model_size_features].to_numpy()
y_array_model_size = sobol_reg[["model_size"]].to_numpy().ravel()

sobol_reg_model_size_group = (
    sobol_reg[model_size_features]
    .round(6)
    .apply(lambda row: "_".join(row.values.astype(str)), axis=1)
)

model_size_model_stem = path.join(model_dir, "sobol_reg_model_size")
model_size_results = rfr_group_mae(
    X_array_model_size, y_array_model_size, sobol_reg_model_size_group, model_size_model_stem, "model_size"
)

MAE for model_size: 7203131.6687 +/- 778089.4128


### runtime

In [48]:
runtime_features = common_features + ["runtime.1"] ## ToDo: runtime or runtime_rank (runtime.1)

X_array_runtime = sobol_reg[runtime_features].to_numpy()
y_array_runtime = sobol_reg[["runtime"]].to_numpy().ravel()

sobol_reg_runtime_group = (
    sobol_reg[runtime_features]
    .round(6)
    .apply(lambda row: "_".join(row.values.astype(str)), axis=1)
)

runtime_model_stem = path.join(model_dir, "sobol_reg_runtime")
runtime_results = rfr_group_mae(
    X_array_runtime, y_array_runtime, sobol_reg_runtime_group, runtime_model_stem, "runtime"
)

MAE for runtime: 70.5667 +/- 13.1031


In [49]:
main_results = {
    "mae": mae_results,
    "rmse": rmse_results,
    "model_size": model_size_results,
    "runtime": runtime_results,
}
with open(path.join(data_dir, "model_metadata.json"), "w") as f:
    json.dump(main_results, f)
    
main_results

{'mae': {'mae': [0.09570551941920749,
   0.09778408166795612,
   0.07742834388223713,
   0.10754968705536598,
   0.06291031172953894],
  'y_pred': [[1.0474287041907535,
    1.134267813083124,
    1.0841832505453575,
    1.0672556696666373,
    1.042669716729676,
    1.0924341708313736,
    0.4749419791982276,
    0.4708075512329325,
    0.5552348275573562,
    0.8956132680693082,
    0.7667366231238202,
    0.9330039752634989,
    0.45414428999303913,
    0.44854202502227475,
    0.4370544748763753,
    0.447200474522955,
    0.382334216293142,
    0.44430961410529124,
    0.4419549275621493,
    0.41667126431814916],
   [0.9220153419556028,
    1.0543371220464375,
    1.0735713814329102,
    0.874739360577308,
    0.4516065732637584,
    0.5896262531681271,
    0.39609978153061687,
    0.47704310885394685,
    0.4610795935552066,
    0.48477912733297857,
    0.39731057861933083,
    0.40568822914073216,
    0.42542491379349046,
    0.46950541072923563,
    0.4253156393119959,
    0.51

In [50]:
model_paths = {
    "mae": mae_model_stem,
    "rmse": rmse_model_stem,
    "model_size": model_size_model_stem,
    "runtime": runtime_model_stem,
}
models = {}
for key, model_path in model_paths.items():
    models[key] = [joblib.load(f"{model_path}_{i}.pkl") for i in range(5)]

with open(path.join(data_dir, "cross_validation_models.pkl"), "wb") as f:
    pickle.dump(models, f)

models

{'mae': [RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13)],
 'rmse': [RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13)],
 'model_size': [RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13)],
 'runtime': [RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13),
  RandomForestRegressor(random_state=13)]}

### code graveyard 

In [51]:
### Ordinal Encoding

# "bias" [False, True]},
# "criterion" ["RobustL1", "RobustL2"]},
# "elem_prop"["mat2vec", "magpie", "onehot"],

# sobol_reg["bias"].replace(["False", "True"], [0,1], inplace=True)
# sobol_reg["criterion"].replace(["RobustL1", "RobustL2"], [0,1], inplace=True)
# sobol_reg["elem_prop"].replace(["mat2vec", "magpie", "onehot"], [0,1,2], inplace=True)
# sobol_reg["hardware"].replace(["2080ti"], [0], inplace=True)


In [52]:
# print("Average MAE for fba_isna_prob",rfr_mae(X_array_fba_isna_prob, y_array_fba_isna_prob,'fba_isna_prob.pkl'))

# load trained model
# loaded_model = joblib.load('fba_isna_prob_model.pkl')

# Save the model
# with open('../models/fba_isna_prob.pkl', 'wb') as f:
#     pickle.dump(model, f)

# # Load the model
# with open('path/to/save/model.pkl', 'rb') as f:
#     loaded_model = pickle.load(f)

In [53]:
# url_sobol_filter = "https://zenodo.org/record/7513019/files/sobol_probability_filter.csv"
# sobol_filter = pd.read_csv(url_sobol_filter)

# url_sobol_reg = "https://zenodo.org/record/7513019/files/sobol_regression.csv"
# sobol_reg = pd.read_csv(url_sobol_reg)

In [54]:
# os.getcwd()
# os.chdir("../data/raw")

# sobol_filter.to_csv('sobol_filter.csv', index=False)

# sobol_reg.to_csv('sobol_reg.csv', index=False)

In [55]:
# read in sobol_regression.csv
# url_sobol_reg = "https://zenodo.org/record/7513019/files/sobol_regression.csv"

In [56]:
# sobol_reg_x = sobol_reg[
#     [
#         "mu1_div_mu3",
#         "mu2_div_mu3",
#         "std1",
#         "std2",
#         "std3",
#         "comp1",
#         "comp2",
#         "num_particles",
#         "safety_factor",
#         "fba_rank",
#         "ls_rank",
#         "fba_time_s_rank",
#         "ls_time_s_rank",
#     ]
# ]

In [57]:
# print(len(sobol_reg_x))
# print(len(fba))

In [58]:
# print(
#     "Average MAE for ls_time_s",
#     rfr_mae(X_array_fba_time_s, y_array_ls_time_s, "sobol_reg_ls_time_s.pkl"),
# )

In [59]:
# # parse data for target "fba_isna_prob"
# fba_isna_prob = sobol_filter["fba_isna_prob"]
# sobolPF_fba_isna_prob = sobol_filter.drop(["ls_isna_prob", "fba_isna_prob"], axis=1)
# fba_isna_prob = fba_isna_prob.to_frame()